In [21]:
import numpy as np
import sys

import torch
from torch.autograd import Variable
import torchvision.models as models
import torch.nn.functional as F
from torch.utils import data, model_zoo
# from model.deeplab_multi import Res_Deeplab
from model.deeplab_single_add_edge import Res_Deeplab

# from dataset.cityscapes_dataset import cityscapesDataSet
from dataset.my_test_dataloader import MyDataSet
from collections import OrderedDict
import os
from PIL import Image

from util.util import get_all_data_loaders, get_config, tensor2im, paint_predict_image, paint_predict_image_np
import matplotlib.pyplot as plt
import torch.nn as nn
import shutil

# config

In [30]:
# DATA_DIRECTORY = './data/Cityscapes/data'
# DATA_LIST_PATH = './dataset/cityscapes_list/val.txt'
# SAVE_PATH = './result/cityscapes''
SAVE_PATH = './training_image_result'
CONFIG_PATH = "./configs/default_decay_beta.yaml"
ORIGIN_SRC_PATH = 
config = get_config(CONFIG_PATH)

IGNORE_LABEL = 255
NUM_CLASSES = 19
# RESTORE_FROM = './snapshots/GTA2Cityscapes_multi/GTA5_30000.pth'
RESTORE_FROM = '/home/xiaosean/data/20181022-AdaptSeg-2DA-Gated-SA-250k-acc4643/snapshots/GTA2Cityscapes_multi/GTA5_250000.pth'
SET='val'

In [8]:
IMG_MEAN = np.array((104.00698793,116.66876762,122.67891434), dtype=np.float32)
palette = [128, 64, 128, 244, 35, 232, 70, 70, 70, 102, 102, 156, 190, 153, 153, 153, 153, 153, 250, 170, 30,
           220, 220, 0, 107, 142, 35, 152, 251, 152, 70, 130, 180, 220, 20, 60, 255, 0, 0, 0, 0, 142, 0, 0, 70,
           0, 60, 100, 0, 80, 100, 0, 0, 230, 119, 11, 32]
zero_pad = 256 * 3 - len(palette)
for i in range(zero_pad):
    palette.append(0)


def colorize_mask(mask):
    # mask: numpy array of the mask
    new_mask = Image.fromarray(mask.astype(np.uint8)).convert('P')
    new_mask.putpalette(palette)

    return new_mask

In [25]:
# data loader
train_loader, target_loader = get_all_data_loaders(config)

GTA_size input size = (1280, 720)
City_size input_size_target = (1024, 512)


In [10]:
model = Res_Deeplab(num_classes=19)

In [11]:
saved_state_dict = torch.load(RESTORE_FROM)
model.load_state_dict(saved_state_dict)

In [12]:
model.eval()
model.cuda(0)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fa

In [13]:
interp = nn.Upsample(size=(1024, 2048), mode='bilinear')

In [22]:
import torchvision.transforms as transforms
def tensor_to_PIL(tensor):
    pilTrans = transforms.ToPILImage()
    image = pilTrans(tensor.cpu().squeeze(0))
    return image

In [26]:
def label_get_edges(t):
    edge = torch.cuda.ByteTensor(t.size()).zero_()
    # t = t.view(1, 1, t.shape[0], t.shape[1])
    # print("edge shape", edge.shape)
    edge[:, :, :, 1:] = edge[:, :, :, 1:] | (t[:, :, :, 1:] != t[:, :, :, :-1])
    edge[:, :, :, :-1] = edge[:, :, :, :-1] | (t[:, :, :, 1:] != t[:, :, :, :-1])
    edge[:, :, 1:, :] = edge[:, :, 1:, :] | (t[:, :, 1:, :] != t[:, :, :-1, :])
    edge[:, :, :-1, :] = edge[:, :, :-1, :] | (t[:, :, 1:, :] != t[:, :, :-1, :])
    return edge.float()


In [34]:
def get_foreground_attention(label):
    foreground_attn = torch.cuda.ByteTensor(label.size()).zero_()
    # choose which label be weakly supervised label
    # 0 > road: 87.89
    # 1 > sidewalk: 31.54
    # 2 > building: 77.69
    # 3 > wall: 23.56
    # 4 > fence: 20.1
    # 5 > pole: 25.49
    # 6 > light: 27.14
    # 7 > sign: 26.57
    # 8 > vegetation: 70.86
    # 9 > terrain: 24.19
    # 10 > sky: 71.61
    # 11 > person: 55.79
    # 12 > rider: 21.21
    # 13 > car: 82.2
    # 14 > truck: 31.67
    # 15 > bus: 44.69
    # 16 > train: 0.9
    # 17 > motocycle: 22.13
    # 18 > bicycle: 25.14

    # ignore background label include 255-ignore label
    # foreground_map = [5, 6, 7, 11, 12, 13, 14, 15, 16, 17, 18, 255]
    foreground_map = [5, 6, 7, 11, 12, 13, 14, 15, 16, 17, 18]
    # foreground_map = [11, 12, 13, 14, 15, 16, 17, 18]

    # foreground_map = [1, 4, 5, 6, 7, 11, 12, 13, 14, 15, 16, 17, 18]
    # foreground_map = [6, 7, 11, 12, 13, 14, 15, 16, 17, 18]
    # foreground_map = [6, 7, 11, 12, 13, 14, 15, 16, 17, 18]

    # choose which label be weakly supervised label

    for value in foreground_map:
        # print(value)
        foreground_attn[label == value] = 1

    return foreground_attn.float()

In [39]:
with torch.no_grad():
    for index, batch in enumerate(train_loader):
        if index == 3:
            print('%d processd' % index)
            break
        image, labels, _, name = batch
        # output1, output2 = model(Variable(image, volatile=True).cuda(gpu0))
        output1, edge = model(Variable(image).cuda(0))

        output = interp(output1).cpu().data[0].numpy()
        edge = interp(edge)
        edge = nn.Sigmoid()(edge).detach()

        output = output.transpose(1,2,0)
        output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

        output_col = colorize_mask(output)
        output = Image.fromarray(output)

        filename = name[0].split('/')[-1]

        
        # save image
        label_name = os.path.join("data", "GTA5", "images", name[0])
        save_name = os.path.join(SAVE_PATH, '%s_input.png' % filename)
        shutil.copyfile(label_name, save_name)

        # save label
        label_name = os.path.join("data", "GTA5", "labels", name[0])
        save_name = os.path.join(SAVE_PATH, '%s_label.png' % filename)
        shutil.copyfile(label_name, save_name)
        
         # save GT edge
        # if no use is not None will be ambiguous
        if labels is not None:
            labels = labels.view(labels.shape[0], 1, labels.shape[1], labels.shape[2])
            tensor_to_PIL(get_foreground_attention(labels.cuda())).save(
                os.path.join(SAVE_PATH, "%s_edge_label.png" % filename))
            # self.tensor_to_PIL(self.get_foreground_attention(labels.cuda())).save('check_output/Image_source_domain_seg/%s_edge_label.png' % self.i_iter)
#         output.save('%s/%s' % (SAVE_PATH, filename))
        output_col.save('%s/%s_color.png' % (SAVE_PATH, filename.split('.')[0]))
        tensor_to_PIL(edge).save('%s/%s_edge.png' % (SAVE_PATH, filename.split('.')[0]))
print("finish!")

/home/xiaosean/anaconda3/envs/xiao/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/home/xiaosean/anaconda3/envs/xiao/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


3 processd
finish!
